# ${\color{purple}\mathbb{NAMES:}}$
1- ${\color{red}Yuval\ Kaver}$, id: 329

2- ${\color{red}Matan\ Ginzburg}$, id: 215

In [1]:
import numpy as np
from numpy.linalg import inv
import scipy

# Bases

In [2]:
def check_basis(a: list):
    '''
    Given a list of n+2 vectors in R^{n+1}, the function checks if this sequence defines a basis of P^n.
    '''
    for i in range(len(a)):
        if np.linalg.det(np.delete(a,i,0)) == 0:  #does the det of a without the vector in the i-th place.
            return False
    return True

a = np.array([[1,0,0],[0,1,0],[0,0,1],[1,1,1]])
b = np.array([[3,1,5],[1,2,7],[2,4,14],[2,6,18]])
c=np.array([[5,6,3],[3,2,4],[9,2,5],[1,4,6]])
d=np.array([[7,5],[22,9],[4,2]])
print("a =",check_basis(a),"  expected = True\n",a)
print("b =",check_basis(b),"  expected = False\n",b)
print("c =",check_basis(c),"  expected = True\n",c)
print("d =",check_basis(d),"  expected = True\n",d)

a = True   expected = True
 [[1 0 0]
 [0 1 0]
 [0 0 1]
 [1 1 1]]
b = False   expected = False
 [[ 3  1  5]
 [ 1  2  7]
 [ 2  4 14]
 [ 2  6 18]]
c = True   expected = True
 [[5 6 3]
 [3 2 4]
 [9 2 5]
 [1 4 6]]
d = True   expected = True
 [[ 7  5]
 [22  9]
 [ 4  2]]


In [3]:
def compute_homography(a:list, b:list):
    '''
    Given two bases of P^n, the function computes the homography A that maps a to b.
    '''
    A = find_matrix_to_standard(a)
    B = find_matrix_to_standard(b)
    inv_A = inv(A) # A^-1
    B_A =B.dot(inv_A) #B*(A^-1)
    return B_A


def find_matrix_to_standard(a:list):
    '''
    finds A that for all a_i -> A*a_i = b_i       where b_i is a vector from the standard basis of P^n
    '''
    a_without_last = np.delete(a,-1,0) #[b_1,b_2,...,b_n+1]
    inv_a = inv(a_without_last.T) #[b1,..,bn+1](-1)
    lambdas = inv_a.dot(a[-1])  # lambdas = a[-1].dot(inv_a) # [b1,....bn+1](-1) * bn+2
    lambdas=np.diag(lambdas)
    A=a_without_last.T.dot(lambdas) #   A = [b_1*lambda,...,b_n+1*lambda]
    return A

a=np.array([[1,2,3],[0,2,4],[0,2,5],[1,2,6]])
b=np.array([[5,6,3],[3,2,4],[9,2,5],[14,24,6]])
B_A = compute_homography(a, b)
for i in range(len(a)):
    b_result = np.dot(B_A,a[i])
    print("b from matrix:",b_result*(a[i][2]/b_result[2]),b[i],"\n")


b from matrix: [5. 6. 3.] [5 6 3] 

b from matrix: [3. 2. 4.] [3 2 4] 

b from matrix: [9. 2. 5.] [9 2 5] 

b from matrix: [14. 24.  6.] [14 24  6] 



# $\mathbb{P}^1$

In [4]:
def cross_ratio(a, b, c, d):
    '''
        Given 4 points in P^{1}, calculate the cross ratio
    '''
    det1 = np.cross(a, b)
    det2 = np.cross(a, c)
    det3 = np.cross(b, d)
    det4 = np.cross(c, d)
    return (det1 / det2) / (det3 / det4)


print("cross for: a = (0, 1), b = (1, 1), c = (2, 1) and d = (1,0), gives:", cross_ratio([0, 1], [1, 1], [2, 1], [1, 0]), "expected = 0.5")
print("cross for: a = (0, 1), b = (1, 1), c = (2, 1) and d = (-1,1), gives:", cross_ratio([0, 1], [1, 1], [2, 1], [-1, 1]), "expected = 0.75")

cross for: a = (0, 1), b = (1, 1), c = (2, 1) and d = (1,0), gives: 0.5 expected = 0.5
cross for: a = (0, 1), b = (1, 1), c = (2, 1) and d = (-1,1), gives: 0.75 expected = 0.75


C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:5: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det1 = np.cross(a, b)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:6: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det2 = np.cross(a, c)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:7: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det3 = np.cross(b, d)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:8: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det4 = np.cross(c, d)


# $\mathbb{P}^2$

In [5]:
def check_if_in_pencil_basis2(p,l1):
    '''
    checks if l1 is in the pencil of lines centered at p
    '''
    return l1.T.dot(p) == 0

In [6]:
def find_basis(p: np.array):
    n = len(p)
    index_of_nonzero = np.where(p != 0)[0]
    zero_indexes = np.where(p == 0)[0]
    basis = np.zeros((n, n))
    
    if len(index_of_nonzero) == 0:
        raise Exception("invalid point")
    elif len(index_of_nonzero) == len(p):  # standard basis
        return np.array([
            [-p[1], p[0], 0],
            [-p[2], 0, p[0]],
            [0, -p[2], p[1]]
        ])
    if len(index_of_nonzero) == 1:  # case [x, 0, 0] -> l1 = [0, 0, 1], l2 = [0, 1, 0] 
        for i, k in enumerate(zero_indexes):
            basis[i] = np.array([1 if i == k else 0 for i in range(3)])
    else:
        basis[0] = (np.array([1 if i == zero_indexes[0] else 0 for i in range(n)]))  # first vector is 0
        basis[1][index_of_nonzero[0]] = -p[index_of_nonzero[-1]]
        basis[1][index_of_nonzero[-1]] = p[index_of_nonzero[0]]

    basis[2] = basis[0] + basis[1]
    return basis

In [7]:
def cross_ratio_pencil(p,l1,l2,l3,l4):
    '''
    Given a point p in P^2 and four lines in the pencil of lines centered at p, 
    the function computes the cross-ratio of the lines. 
    '''
    l = np.array([l1,l2,l3,l4])
    l_new = np.zeros((4,2))
    for i in range(len(l)):
        if not check_if_in_pencil_basis2(p,l[i]):#checks if l_i is in the pencil of lines centered at p
            raise Exception(f"{l[i]} is not in p")
    basis = (np.delete(find_basis(p),2,axis=0)).T
    for i in range(3):
        if np.linalg.det(np.delete(basis,i,axis=0)) != 0:
            basis = np.delete(basis,i,axis=0)
            l = np.delete(l,i,axis=1)
            break
    for i in range(len(l)):
        l_new[i] = np.linalg.solve(basis,l[i].T)
    return cross_ratio(*l_new)


p=np.array([6,2,3])
l1=np.array([-1,0,2])
l2=np.array([2,-3,-2])
l3=np.array([0,-3,2])
l4=np.array([1,-3,0])
print(cross_ratio_pencil(p,l1,l2,l3,l4))

-1.0


C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:5: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det1 = np.cross(a, b)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:6: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det2 = np.cross(a, c)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:7: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det3 = np.cross(b, d)
C:\Users\matan\AppData\Local\Temp\ipykernel_22388\3278308293.py:8: DeprecationWarning: Arrays of 2-dimensional vectors are deprecated. Use arrays of 3-dimensional vectors instead. (deprecated in NumPy 2.0)
  det4 = np.cross(c, d)


# Numerical Homographies in $\mathbb{P}^2$

In [8]:
n = 100
p = np.random.randint(2,10)*np.random.randn(n,2) + np.random.randint(10,100)
p = np.hstack((p,np.ones((n,1))))
H = np.random.randn(3,3)
qq = (H@p.T).T
q = (qq.T/qq.T[2:3,:]).T
#q = q[:,0:2]



In [9]:
p

array([[32.46547282, 24.56470639,  1.        ],
       [24.91598059, 25.79849659,  1.        ],
       [30.57698704, 26.01300294,  1.        ],
       [31.13734907, 26.59465811,  1.        ],
       [29.62754943, 31.39008773,  1.        ],
       [34.40546725, 27.66903791,  1.        ],
       [25.83745952, 38.98409134,  1.        ],
       [27.09954133, 34.16114014,  1.        ],
       [30.10227569, 24.63911053,  1.        ],
       [31.83462435, 35.32609716,  1.        ],
       [31.90361575, 24.04828628,  1.        ],
       [20.52665575, 30.05890284,  1.        ],
       [29.09298562, 22.99313695,  1.        ],
       [29.58644474, 29.61034155,  1.        ],
       [36.72270648, 26.07408607,  1.        ],
       [25.98711329, 28.55164893,  1.        ],
       [25.36901287, 32.11585838,  1.        ],
       [31.09987699, 31.91305093,  1.        ],
       [30.83749742, 28.86361434,  1.        ],
       [34.94894355, 41.17267039,  1.        ],
       [31.72272007, 29.6159825 ,  1.   

In [10]:
q

array([[-1.36446318, -0.87483566,  1.        ],
       [-1.49137261, -0.49803773,  1.        ],
       [-1.40665638, -0.74539513,  1.        ],
       [-1.40721386, -0.74140402,  1.        ],
       [-1.49149434, -0.47446611,  1.        ],
       [-1.38214185, -0.81117196,  1.        ],
       [-1.66016107,  0.05526512,  1.        ],
       [-1.57114581, -0.22743506,  1.        ],
       [-1.39399606, -0.78831668,  1.        ],
       [-1.5073323 , -0.41481959,  1.        ],
       [-1.36400442, -0.87836267,  1.        ],
       [-1.65992058,  0.02007682,  1.        ],
       [-1.38379495, -0.82569266,  1.        ],
       [-1.46871044, -0.54844463,  1.        ],
       [-1.33943397, -0.94230058,  1.        ],
       [-1.51333743, -0.42067737,  1.        ],
       [-1.57674797, -0.21809985,  1.        ],
       [-1.47627673, -0.51798732,  1.        ],
       [-1.44099572, -0.63319569,  1.        ],
       [-1.52898157, -0.33494912,  1.        ],
       [-1.43874286, -0.63717552,  1.   

In [11]:
p = np.array([[384.37011284557997,404.98615093770684],
             [470.66420458240714,272.46308148472224],
             [742.6446901458,324.08543993443135],
             [732.6284116406325,492.05072563647]])


P = np.array([[0,0],
              [1,0],
              [1,1],
              [0,1]])

In [12]:
P.shape

(4, 2)

In [13]:
def svd_computation_homography(p: np.ndarray, q: np.ndarray):
    '''
    Given two nx2 arrays of affine points, the function computes the homography between these two sets of points by the SVD method. 
    Output: H a 3x3 matrix
    '''
    M = np.zeros((2*len(p)-2,9))
    for i in range(len(p)-1):
        M[2*i] = [p[i][0]*1,p[i][1]*1,1*1,0,0,0,-p[i][0]*q[i][0],-p[i][1]*q[i][0],-1*q[i][0]]
        M[2*i+1] = [0,0,0,p[i][0],p[i][1],1,-p[i][0]*q[i][1],-p[i][1]*q[i][1],-q[i][1]]
    MTM = M.T.dot(M)
    U , _ , __= np.linalg.svd(MTM)
    A = np.reshape(U.T[-1],(3,3))
    return A

A = svd_computation_homography(p,q)
print(f"A = {A*(H[0][0]/A[0][0])},\n H = {H}") # A is multiplied by the first element of H divided by the first element of A

A = [[ 1.00436266e+00 -5.90982189e+00 -5.00073060e+02]
 [ 2.46763131e-01 -7.84391994e+00  1.47417835e+03]
 [-6.52168319e-01  5.15652940e+00  1.29790415e-02]],
 H = [[ 1.00436266  0.77553303  0.3701197 ]
 [ 2.1741642  -1.44460715 -1.74080037]
 [-1.05114838 -0.20122354  0.93830192]]


In [16]:
def normalization(p: np.ndarray):
    '''
    Given a nx2 array of affine points, the function computes the normalizing homography of these points.
    Output: T a 3x3 matrix (which lies in Aff2).
    '''
    avg_x, avg_y, _ = np.mean(p,axis=0)
    std_x, std_y, _ = np.std(p,axis=0)
    T = np.array([
        [1/std_x,0,-avg_x/std_x],
        [0,1/std_y,-avg_y/std_y],
        [0,      0,          1]
    ])
    return T

T = normalization(p)
p_normalized = np.array(p)
for i in range(len(p_normalized)):
    p_normalized[i] = T@p_normalized[i]
avg_x, avg_y, _ = np.mean(p_normalized,axis=0)
print(f"avg_x = {avg_x.round(5)}\navg_y = {avg_y.round(5)}")
std_x, std_y, _ = np.std(p_normalized,axis=0)
print(f"std_x = {std_x.round(5)}\nstd_y = {std_y.round(5)}")


ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
def homography(p: np.ndarray, q: np.ndarray):
    '''
    Given two nx2 arrays of affine points, the function computes the homography between these two sets of points by normalizations and the SVD method. 
    Output: H a 3x3 matrix
    '''
    T1 = normalization(p)
    T2 = normalization(q)
    p_normalized = np.array(p)
    q_normalized = np.array(p)
    for i in range(len(p)):
        p_normalized[i] = T1@p[i] 
        q_normalized[i] = T2@q[i]    
    B = svd_computation_homography(p_normalized,q_normalized)
    return (inv(T2)).dot(B.dot(T1))
A = homography(p,q)
A *= H[0][0]/A[0][0] # A is multiplied by the first element of H divided by the first element of A
print(f"A = {A},\n H = {H}") 

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
def geometric_error(H: np.ndarray,p: np.ndarray, q: np.ndarray):
    '''
    Given an homography H and two nx2 arrays of affine points, the function computes the residual geometric error. 
    Output: float
    '''
    A = np.reshape(H,(3,3))
    Error = 0    
    for i in range(len(p)):

        u_i = q[i][0]
        v_i = q[i][1]
        
        A_pi_x = (A[0].T@p[i])/(A[2].T@p[i])
        A_pi_y = (A[1].T@p[i])/(A[2].T@p[i])
        
        Error += (A_pi_x - u_i)**2 + (A_pi_y - v_i)**2

    return Error
print(geometric_error(A,p,q))

9.423575217886745e-31


In [ ]:
def refine_homography(H: np.ndarray,p: np.ndarray, q: np.ndarray):
    '''
    Given an homography H and two nx2 arrays of affine points, the function refines the homography by a non-linear minimization of the geometric error. 

    See: scipy.optimize (least_square: Newton with Levenberg-Marquardt).

    Output: a 3x3 matrix
    '''
    A = np.matrix.flatten(H)
    # return scipy.optimize.minimize(geometric_error,x0=A,args=(p,q))
    return scipy.optimize.least_squares(geometric_error,x0=A,args=(p,q))

B = np.reshape(refine_homography(A,p,q).x,(3,3))
print(geometric_error(A,p,q))
print(geometric_error(B,p,q))

9.423575217886745e-31
9.423575217886745e-31


# Conics in $\mathbb{P}^2$

In [ ]:
from enum import Enum
class ConicInformation(Enum):
    DOUBLE_LINE = 1
    TWO_LINES = 2
    PARABOLA = 3
    HYPERBOLA = 4
    ELLIPSE = 5

def conic_classification(C):
    '''
    Given a symmatric 3x3 matrix, the function returns the sort of conic it defines in P^2. 
    Returns a variable of type ConicInformation
    '''
    if np.any(C.T != C):
        raise AttributeError("C must be symetric")
    if np.linalg.matrix_rank(C)==1:      #Double line!
        return ConicInformation.DOUBLE_LINE   
    elif np.linalg.matrix_rank(C)==2:    #Two lines!    
        return  ConicInformation.TWO_LINES
    root_of_C = (2*C[0,1])**2 - 4*C[0][0]*C[1][1]
    if root_of_C == 0:
        return ConicInformation.PARABOLA
    if root_of_C > 0:
        return ConicInformation.HYPERBOLA
    if root_of_C < 0:
        return ConicInformation.ELLIPSE
    
C = np.array([[2,9,-1],[9,7,-3],[-1,-3,4]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(4).name}")
C = np.array([[7,42,-5],[42,257,-211],[-5,-211,41]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(5).name}")
C = np.array([[1,2,3],[2,4,6],[3,6,43]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(2).name}")
C = np.array([[1,1,1],[1,1,1],[1,1,1]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(1).name}")
C = np.array([[1,-1,-1],[-1,1,-3],[-1,-3,4]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(3).name}")
C = np.array([[1,0,0],[0,0,-1/2],[0,-1/2,0]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(3).name}")
C = np.array([[0,-1/2,0],[-1/2,0,0],[0,0,1]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(4).name}")
C = np.array([[420,0,0],[0,69,0],[0,0,-1]])
print(f"we classified: {conic_classification(C).name}, the conic is {ConicInformation(5).name}")

we classified: HYPERBOLA, the conic is HYPERBOLA
we classified: ELLIPSE, the conic is ELLIPSE
we classified: TWO_LINES, the conic is TWO_LINES
we classified: DOUBLE_LINE, the conic is DOUBLE_LINE
we classified: PARABOLA, the conic is PARABOLA
we classified: PARABOLA, the conic is PARABOLA
we classified: HYPERBOLA, the conic is HYPERBOLA
we classified: ELLIPSE, the conic is ELLIPSE


In [ ]:
def parameters_l(l):
    ortogonal_base = np.zeros((3,3))
    ortogonal_base[0] = l
    p = scipy.linalg.null_space(ortogonal_base).T
    return p
    
l = np.array([8,4,2])
p_q = parameters_l(l)
print(f"p = {p_q[0]/p_q[0][2]}\nq = {p_q[1]/p_q[1][2]}")
print("\nl^T*p = ",l.T@p_q[0],"\nl^T*q = ",(l.T@p_q[1]).round(10))

p = [ 0.  -0.5  1. ]
q = [-1.25  2.    1.  ]

l^T*p =  0.0 
l^T*q =  0.0


In [ ]:
from numpy import complex128


def intersection_conic_line(C,l):
    '''
    Given a non-degenarated conic C and a line l in P^2, the function returns the two intersection points of l with C (with complex coordinates).  
    '''
    # One must turn the line into a parametric representation and then look for the intersection point with the conic. 
    p , q = parameters_l(l)
    #the line is represented as p + (q - p)t. we can divide by t and get: p/t + q - p => (1-t/t)p + q , we can write w = (1-t/t) and get q + w * p. we will refer to w as t from now on
    # so point in line is: x(t) = q1 + p1 * t,q2 + p2 * t,q3 + p3 * t)      where p = (p1,p2,p3) and q = (q1,q2,q3)
    #if we do the equation x^T C x, we get a quadratic equation with t which we can solve
    #we will write it as a*t^2 + b*t + c. 
    # a = sum(p_i*p_j*C_ij) , for all i and j = 1,2,3
    # b = sum(q_i*p_j*C_ij + q_j*p_i*C_ij) , for all i and j = 1,2,3
    # c = sum(q_i*q_j*C_ij) , for all i and j = 1,2,3
    a = 0
    b = 0
    c = 0
    if (p.T@C@p).round(5) == 0: #if the point p in on the conic, we will move it. so the code wont be over complicated.
        p = p + 0.1*q
        if (p.T@C@p).round(5) == 0: # if its on the conic again we will move it again. if we wanted to skip this we could have checked if p is in the conic and then get a linear equation for the other tangent point
            p = p + 0.1*q
    for i in range(3):
        for j in range(3):
            a += p[i]*p[j]*C[i][j]
            b += q[i]*(p[j])*C[i][j] + q[j]*(p[i])*C[i][j]
            c += q[i]*q[j]*C[i][j]
    t1 = (-b + np.sqrt(b**2-4*a*c,dtype=complex128))/(2*a) #parameters
    t2 = (-b - np.sqrt(b**2-4*a*c,dtype=complex128))/(2*a)
    p1 = q + p*t1  #intersection points
    p2 = q + p*t2
    return p1,p2
C = np.array([[7,42,-5],[42,257,-211],[-5,-211,41]])
l = np.array([1,12,3])
p1 , p2 = intersection_conic_line(C,l)
print(f"p1 = {p1/p1[2]} \np2 = {p2/p2[2]}")
print(f"pT*C*p = {(p1.T@C@p1).round(10)}\nqT*C*q = {(p2.T@C@p2).round(10)}")

p1 = [-4.16731518-8.58598203j  0.09727626+0.7154985j   1.        +0.j        ] 
p2 = [-4.16731518+8.58598203j  0.09727626-0.7154985j   1.        -0.j        ]
pT*C*p = -0j
qT*C*q = 0j


In [ ]:
def tangency_points(C,p):
    '''
    Given a non-degenerated conic C and a point p, the function returns the two tangency points of tangent lines to C through p.  
    Returns [q1,q2].
    '''
    cp = C@p # polar line of P!!!!!
    if p.T@C@p==0:        
        return cp
    return intersection_conic_line(C,cp)
C = np.array([[1,2,3],[2,4,6],[3,6,43]])
p = np.array([5,3,2])
p , q = tangency_points(C,p)
print(f"p = {p} \nq = {q}")
print(f"pT*C*p = {(p.T@C@p).round(10)}\nqT*C*q = {(q.T@C@q).round(10)}")

p = [ 0.99447245+0.j -0.16361311+0.j -0.00228776+0.j] 
q = [0.99447245+0.j 0.51029095+0.j 0.57773913+0.j]
pT*C*p = 0j
qT*C*q = 0j


C:\Users\yuval\AppData\Local\Temp\ipykernel_127332\226279080.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  ortogonal_base[0] = l
